In [1]:
from __future__ import division
import numpy as np
from glob import glob
from os.path import join, basename
from nipype.interfaces import afni

In [2]:
data_dir = '/scratch/PSB6351_2017/students/flannery/output/output_week7/'
mask_dir = '/scratch/PSB6351_2017/ds008_R2.0.0/preproc/anat_masks/'

smoothing_kernels = ['sm3', 'sm6', 'sm9', 'sm12']

f_acfs = np.empty((len(smoothing_kernels), 4))
for k, sm in enumerate(smoothing_kernels):
    sm_data_dir = join(data_dir, sm)
    
    subjects = sorted([basename(s) for s in glob(join(sm_data_dir, 'sub-*'))])
    s_acfs = np.empty((len(subjects), 4))
    for i, s in enumerate(subjects):
        mask_file = join(mask_dir, s, 'masks/_subject_id_{0}/wb_warped.nii.gz'.format(s))
        
        runs_dir = join(sm_data_dir, s, 'preproc/func/smoothed_highpassed/')
        run_files = sorted([basename(r) for r in glob(join(runs_dir, 'corr*run*.nii.gz'))])

        r_acfs = np.empty((len(run_files), 4))
        for j, r in enumerate(run_files):
            f = join(runs_dir, r)
            fwhm = afni.FWHMx(in_file=f, acf=True, detrend=True, mask=mask_file)
            res = fwhm.run()
            r_acfs[j, :] = np.array(res.outputs.acf_param)
            if sm=='sm12' and s=='sub-01' and j==0:
                dude = r_acfs[j, :]
        s_acf = np.mean(r_acfs, axis=0)
        s_acfs[i, :] = s_acf
    f_acf = np.mean(s_acfs, axis=0)
    f_acfs[k, :] = f_acf

170327-11:32:45,647 interface INFO:
	 stderr 2017-03-27T11:32:45.647495:++ 3dFWHMx: AFNI version=AFNI_16.3.18 (Dec 22 2016) [64-bit]
170327-11:32:45,650 interface INFO:
	 stderr 2017-03-27T11:32:45.647495:++ Authored by: The Bob
170327-11:32:45,652 interface INFO:
	 stderr 2017-03-27T11:32:45.647495:++ Number of voxels in mask = 26814
170327-11:32:47,182 interface INFO:
	 stderr 2017-03-27T11:32:47.182136:++ detrending start: 13 baseline funcs, 176 time points
170327-11:32:48,198 interface INFO:
	 stderr 2017-03-27T11:32:48.198511: + detrending done (0.00 CPU s thus far)
170327-11:32:48,702 interface INFO:
	 stderr 2017-03-27T11:32:48.702354:++ Output dataset ./corr_sub-01_task-conditionalstopsignal_run-01_bold_bet_afni_mask_bp_detrend+orig.BRIK
170327-11:32:48,704 interface INFO:
	 stderr 2017-03-27T11:32:48.702354:++ start FWHM calculations
170327-11:32:48,705 interface INFO:
	 stderr 2017-03-27T11:32:48.702354: + FWHM done (0.00 CPU s thus far)
170327-11:32:48,708 interface INFO:
	 

In [3]:
print f_acfs[:, :-1]

[[  0.75381499   2.66475921   7.12850443]
 [  0.64296675   3.20177098   9.20409329]
 [  0.51202263   5.1836675   11.81813738]
 [  0.5384084    7.1807349   15.04756167]]


In [5]:
print('Using 2-sided, NN=2 (edge-connected), p=0.01, a=0.05')
print('3mm min: 14')
print('6mm min: 24')
print('9mm min: 65')
print('12mm min: 111')

Using 2-sided, NN=2 (edge-connected), p=0.01, a=0.05
3mm min: 14
6mm min: 24
9mm min: 65
12mm min: 111
